In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
#importo csv
df_reviews = pd.read_csv('datos/user_reviews_clean.csv', encoding='utf-8',sep="\t")

In [5]:
df_games = pd.read_csv('datos/steam_games_clean.csv', encoding='utf-8')

In [6]:
df_items = pd.read_csv('datos/user_items_clean.csv', encoding='utf-8')

In [7]:
# Se hace un subconjunto con los que serán necesarios para la consulta
contenido_free_items = df_games[['price', 'release_year', 'developer', 'id']]
# Se renombra la columna para luego poder unirla con items
contenido_free_items = contenido_free_items.rename(columns={'id':'item_id'})

In [36]:
contenido_free_items.head(2)

,price,release_year,developer,item_id
0,4.99,2018.0,Kotoshiro,761140.0
1,4.99,2018.0,Kotoshiro,761140.0


In [8]:
# Se toman las columnas que serán necesarias
df_price = df_games[['price', 'id']]
# Se renombra la columna para luego poder unirla con items
df_price = df_price.rename(columns={'id':'item_id'})

In [9]:
# Se toman las columnas que serán necesarias
df_price_items = df_items[['items_count', 'user_id', 'item_id']]
df_price_items.head(2)

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20


In [10]:
# Se hace un merge entre las dos tablas anteriores
df_price_items = df_price_items.merge(df_price, on='item_id',how='left')
df_price_items.head()

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99


In [11]:
# Se hace una revision de nulos para los precios 
df_price_items['price'].isna().sum()

937657

In [14]:
# Los precios que tienen nulos, como no tienen asignado un precio se imputaran ceros 
df_price_items['price'] = df_price_items['price'].fillna(0)

In [15]:
df_price_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
10814956,7,76561198329548331,388490,0.00
10814957,7,76561198329548331,521570,0.00
10814958,7,76561198329548331,521570,0.00
10814959,7,76561198329548331,521570,0.00


In [16]:
# elimina la columna items_id
df_price_items = df_price_items.drop('item_id', axis=1)

In [17]:
# Agrupa por usuario y la sumna de price por usuario
df_price_user = df_price_items.groupby('user_id')['price'].sum().reset_index()
df_price_user

,user_id,price
0,--000--,1566.35
1,--ace--,363.57
2,--ionex--,169.86
3,-2SV-vuLB-Kg,942.80
4,-404PageNotFound-,3484.47
...,...,...
70907,zzonci,59.94
70908,zzoptimuszz,369.92
70909,zzydrax,209.88
70910,zzyfo,1417.12


In [18]:
# Se retoman las columnas necesarias
df_items_count = df_price_items[['items_count','user_id']]
# Y se eliminan los duplicados para poder unirlos con el df_price_user
df_items_count = df_items_count.drop_duplicates(subset='user_id', keep='first')

In [46]:
df_items_count

,items_count,user_id
0,277,76561197970982479
498,888,js41637
2316,137,evcentric
2596,328,Riot-Punch
3139,541,doctr
...,...,...
10813369,321,76561198320136420
10814366,4,ArkPlays7
10814381,22,76561198323066619
10814407,177,76561198326700687


In [19]:
# Se tiene el mismo numero de filas, se hace el merge
df_user_data = df_items_count.merge(df_price_user, on='user_id', how='right')
df_user_data

,items_count,user_id,price
0,58,--000--,1566.35
1,44,--ace--,363.57
2,23,--ionex--,169.86
3,68,-2SV-vuLB-Kg,942.80
4,149,-404PageNotFound-,3484.47
...,...,...,...
70907,5,zzonci,59.94
70908,61,zzoptimuszz,369.92
70909,13,zzydrax,209.88
70910,84,zzyfo,1417.12


In [20]:
# Se seleccionan las que serán utiles para Horas por usuario
df_user_playtime = df_items[['playtime_forever', 'user_id', 'item_id']]
df_user_playtime.head()

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50


In [21]:
# Se va a necesitar tambien items y genero
df_items_genre = df_games[['genres','id','release_year']]
df_items_genre = df_items_genre.rename(columns={'id':'item_id'})
df_items_genre

,genres,item_id,release_year
0,Action,761140.0,2018.0
1,Casual,761140.0,2018.0
2,Indie,761140.0,2018.0
3,Simulation,761140.0,2018.0
4,Strategy,761140.0,2018.0
...,...,...,...
71549,Indie,610660.0,2018.0
71550,Racing,610660.0,2018.0
71551,Simulation,610660.0,2018.0
71552,Casual,658870.0,2017.0


In [22]:
# Se hace un merge de los data frames anteriores
df_user_time_genre = df_user_playtime.merge(df_items_genre, on='item_id')
df_user_time_genre

,playtime_forever,user_id,item_id,genres,release_year
0,6,76561197970982479,10,Action,2000.0
1,0,js41637,10,Action,2000.0
2,0,Riot-Punch,10,Action,2000.0
3,93,doctr,10,Action,2000.0
4,108,corrupted_soul,10,Action,2000.0
...,...,...,...,...,...
9877299,164,76561198107283457,354280,Indie,2016.0
9877300,164,76561198107283457,354280,Simulation,2016.0
9877301,0,inven,433920,Adventure,2016.0
9877302,0,inven,433920,Indie,2016.0


In [23]:
# Se hace la agrupacion 
df_user_time_genre = df_user_time_genre.groupby(['genres', 'user_id', 'release_year'])['playtime_forever'].sum().reset_index()
# Se convierte a horas el tiempo jugado
df_user_time_genre['playtime_hrs'] = df_user_time_genre['playtime_forever']/60
# Se elimina la columna anterior de tiempo jugado
df_user_time_genre = df_user_time_genre.drop('playtime_forever', axis=1)
df_user_time_genre

,genres,user_id,release_year,playtime_hrs
0,Action,--000--,2009.0,88.816667
1,Action,--000--,2010.0,0.366667
2,Action,--000--,2011.0,108.700000
3,Action,--000--,2012.0,1822.433333
4,Action,--000--,2013.0,6.050000
...,...,...,...,...
3436430,Web Publishing,zepavil,2015.0,150.166667
3436431,Web Publishing,zeshirky,2007.0,0.016667
3436432,Web Publishing,zevlupine,2012.0,0.066667
3436433,Web Publishing,zilaman,2013.0,0.150000


In [24]:
# Se cargan los datasets con las columnas que serán necesarias
df_dev = df_games[['developer','id','release_year']]
df_items_conect = df_items[['item_id','user_id']]
df_reviews_sentiment = df_reviews[['sentiment_analysis','reviews_recommend','user_id']]

In [25]:
# Se prepara para hacer el merge
df_dev = df_dev.rename(columns={'id':'item_id'})

In [26]:
# Se aplica el merge sobre items
df_dev_items = df_dev.merge(df_items_conect,on='item_id')

In [27]:
df_dev_items

,developer,item_id,release_year,user_id
0,Stainless Games Ltd,282010.0,1997.0,UTNerd24
1,Stainless Games Ltd,282010.0,1997.0,I_DID_911_JUST_SAYING
2,Stainless Games Ltd,282010.0,1997.0,76561197962104795
3,Stainless Games Ltd,282010.0,1997.0,r3ap3r78
4,Stainless Games Ltd,282010.0,1997.0,saint556
...,...,...,...,...
9877299,Valve,80.0,2004.0,76561198273508956
9877300,Valve,80.0,2004.0,76561198282090798
9877301,Valve,80.0,2004.0,943525
9877302,Valve,80.0,2004.0,76561198283312749


In [28]:
# se procede a unir con el ultimo dataframe
df_reviews_dev = df_reviews_sentiment.merge(df_dev_items,on='user_id')

In [29]:
#reviso duplicados
df_reviews_dev.duplicated().sum()

8552832

In [30]:
#elimino duplicados
df_reviews_dev.drop_duplicates(inplace=True)

In [31]:
#vulvo a contar duplicados
df_reviews_dev.duplicated().sum()

0

In [32]:
df_reviews_dev

,sentiment_analysis,reviews_recommend,user_id,developer,item_id,release_year
0,2,True,76561197970982479,Valve,70.0,1998.0
1,2,True,76561197970982479,Arkane Studios,1700.0,2002.0
2,2,True,76561197970982479,"2K Boston,2K Australia",7670.0,2007.0
4,2,True,76561197970982479,"Epic Games, Inc.",13250.0,1998.0
5,2,True,76561197970982479,Oddworld Inhabitants,15710.0,1998.0
...,...,...,...,...,...,...
11854177,2,True,mijiperki,Bohemia Interactive,33930.0,2010.0
11854180,2,True,mijiperki,Ubisoft Montreal,33230.0,2010.0
11854182,2,True,mijiperki,Avalanche Studios,8190.0,2010.0
11854184,2,True,mijiperki,Nadeo,11020.0,2008.0


In [33]:
# Se elimina la columna que sirvio como union
df_reviews_dev = df_reviews_dev.drop(columns='item_id', axis=1)

In [34]:
df_reviews_dev.columns

Index(['sentiment_analysis', 'reviews_recommend', 'user_id', 'developer',
       'release_year'],
      dtype='object')

In [35]:
# Ahora solo dejamos los registros con review igual a true y con analisis de sentimiento = 2
df_best_developer_year = df_reviews_dev[(df_reviews_dev['reviews_recommend'] == True) & (df_reviews_dev['sentiment_analysis'] == 2)]

In [36]:
df_best_developer_year.head()

,sentiment_analysis,reviews_recommend,user_id,developer,release_year
0,2,True,76561197970982479,Valve,1998.0
1,2,True,76561197970982479,Arkane Studios,2002.0
2,2,True,76561197970982479,"2K Boston,2K Australia",2007.0
4,2,True,76561197970982479,"Epic Games, Inc.",1998.0
5,2,True,76561197970982479,Oddworld Inhabitants,1998.0


In [38]:
#cambio a tipo int release_year
df_best_developer_year = df_best_developer_year.copy()
df_best_developer_year['release_year'] = df_best_developer_year['release_year'].fillna(0).astype(int)

In [39]:
df_best_developer_year.head()

,sentiment_analysis,reviews_recommend,user_id,developer,release_year
0,2,True,76561197970982479,Valve,1998
1,2,True,76561197970982479,Arkane Studios,2002
2,2,True,76561197970982479,"2K Boston,2K Australia",2007
4,2,True,76561197970982479,"Epic Games, Inc.",1998
5,2,True,76561197970982479,Oddworld Inhabitants,1998


In [40]:
# Se crea el data frame para developer_review_analysis llamado: df_dev_rev_analysis
df_dev_rev_analysis = df_reviews_dev[['developer','sentiment_analysis']]
df_dev_rev_analysis

,developer,sentiment_analysis
0,Valve,2
1,Arkane Studios,2
2,"2K Boston,2K Australia",2
4,"Epic Games, Inc.",2
5,Oddworld Inhabitants,2
...,...,...
11854177,Bohemia Interactive,2
11854180,Ubisoft Montreal,2
11854182,Avalanche Studios,2
11854184,Nadeo,2


In [41]:
df_dev_rev_analysis['sentiment_analysis'].nunique()

3

In [42]:
# convierto a parquet contenido_free_items
contenido_free_ruta = 'datos/contenido_free_items.parquet'
contenido_free_items.to_parquet(contenido_free_ruta, index=False)

In [43]:
# convierto a parquet df_user_data
user_data_ruta = 'datos/user_data_price.parquet'
df_user_data.to_parquet(user_data_ruta, index=False)

In [44]:
#convierto a parquet user_for_genre
user_time_genre_ruta = 'datos/user_for_genre.parquet'
df_user_time_genre.to_parquet(user_time_genre_ruta, index=False)

In [45]:
# convierto a parquet best_developer_year
developer_year_ruta = 'datos/best_developer_year.parquet'
df_best_developer_year.to_parquet(developer_year_ruta, index=False)

In [46]:
# convierto a parquet developer_review_analysis
dev_rev_analysis_ruta = 'datos/dev_rev_analysis.parquet'
df_dev_rev_analysis.to_parquet(dev_rev_analysis_ruta, index=False)

In [ ]:
#convierto a parquet user_reviews_clean
user_reviews_ruta = 'datos/user_reviews_clean.parquet'
df_reviews.to_parquet(user_reviews_ruta, index=False)